In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [3]:
import torch

# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla T4


In [4]:
!pip install transformers

     |████████████████████████████████| 778kB 13.6MB/s 
     |████████████████████████████████| 1.1MB 59.4MB/s 
     |████████████████████████████████| 890kB 61.6MB/s 
     |████████████████████████████████| 3.0MB 54.0MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=84993cb0ba603d83b1d0c00e65a2cce37e95d51caa842b9dca65058fcb0aba81
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [5]:
from transformers import BertTokenizer

# Load the BERT tokenizer.
print('Loading BERT tokenizer...')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

Loading BERT tokenizer...


In [6]:
import pandas as pd

train_df = pd.read_csv('/content/gdrive/My Drive/Multi_Model_Emotion_Detection/DailyDialog/CSV/train.csv', index_col=0)
test_df = pd.read_csv('/content/gdrive/My Drive/Multi_Model_Emotion_Detection/DailyDialog/CSV/test.csv', index_col=0)
val_df = pd.read_csv('/content/gdrive/My Drive/Multi_Model_Emotion_Detection/DailyDialog/CSV/val.csv', index_col=0)

In [7]:
dataset_df = pd.concat([train_df, test_df, val_df])

In [8]:
len(dataset_df)

102979

In [9]:
df_list = list()
for emotion, emotion_df in dataset_df.groupby(by=['emotion']):
  samples_count = min(2500, len(emotion_df))
  df_list.append(emotion_df.sample(n=samples_count))

In [10]:
sampled_df = pd.concat(df_list)

In [11]:
sampled_df

,emotion,act,text,topic,label
35217,anger,inform,"I know , but it is hard for me to get used to ...",relationship,0
4722,anger,question,How could you ? I reminded you just this morni...,relationship,0
70917,anger,directive,You can ' t just throw me out on the street !,ordinary_life,0
70750,anger,directive,"But what ? You always act like this , when you...",work,0
55447,anger,commissive,That's not nice .,relationship,0
...,...,...,...,...,...
30941,surprise,inform,"Really ? I've already opened this one . Say , ...",relationship,1
38359,surprise,inform,"Oh dear , my weight has gone up again .",ordinary_life,1
1363,surprise,question,You didn ' t ?,politics,1
9693,surprise,question,You mean get married now ? On the plane ?,attitude_and_emotion,1


In [26]:
from transformers import BertForSequenceClassification, AdamW, BertConfig

# Load BertForSequenceClassification, the pretrained BERT model with a single 
# linear classification layer on top. 
model = BertForSequenceClassification.from_pretrained(
    "/content/gdrive/My Drive/Multi_Model_Emotion_Detection/BERT Models/DailyDialog+MELD+IEMOCAP/4/epoch_3", # Use the 12-layer BERT model, with an uncased vocab.
    num_labels = 5, # The number of output labels--2 for binary classification.
                    # You can increase this for multi-class tasks.   
    output_attentions = False, # Whether the model returns attentions weights.
    output_hidden_states = True, # Whether the model returns all hidden-states.
)

# Tell pytorch to run this model on the GPU.
model.cuda()

print('Model Ready')

Model Ready


In [27]:
def get_embeddings(utterance):
  input_ids = tokenizer.encode(utterance, add_special_tokens=True)
  input_ids = torch.LongTensor(input_ids).unsqueeze(0).to(device)
  _, hidden_states = model(input_ids)
  embedding = hidden_states[-1][0][0]
  return embedding.cpu().detach().numpy()

In [28]:
sampled_df.columns

Index(['emotion', 'act', 'text', 'topic', 'label', 'Text Embeddings', 'ID'], dtype='object')

In [29]:
sampled_df['Text Embeddings'] = sampled_df['text'].apply(get_embeddings)

In [30]:
EMOTION_IDS = {
    0: 'anger/disgust',
    1: 'fear/surprise',
    2: 'happiness',
    3: 'neutral',
    4: 'sadness'
}

In [31]:
def get_id(self, dataset):
  global idx
  idx += 1
  return "%s-%s-%s" % (EMOTION_IDS[self.label], dataset, idx)

In [32]:
idx = 0
sampled_df['ID'] = sampled_df.apply(lambda row: get_id(row, 'train'), axis=1)

In [33]:
import pickle
pickle.dump(sampled_df[['ID', 'Text Embeddings']], open('/content/gdrive/My Drive/Multi_Model_Emotion_Detection/Embeddings/DailyDialog Embeddings/DailyDialog+MELD+IEMOCAP/4/text_embeddings.pkl', 'wb'))